In [ ]:
import os
import sys
sys.path.append('../../')

import django
import marslab.spectops as ops
import matplotlib.font_manager as mplf
import matplotlib.pyplot as plt
import pandas as pd
from marslab.compat.xcam import DERIVED_CAM_DICT
from marslab.imgops.pltutils import set_label
from sklearn.linear_model import LogisticRegression

from correlate import (
    correlation_matrix, explode_sequential, make_corr_chart
)


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()
from plotter.spectrum_ops import filter_df_from_queryset
import plotter.models
from multidex_utils import model_metadata_df

%matplotlib qt
%matplotlib qt

In [ ]:
instrument = "ZCAM"
spec_model = plotter.models.INSTRUMENT_MODEL_MAPPING[instrument]
metadata_df = model_metadata_df(spec_model)
filter_info = DERIVED_CAM_DICT[instrument]["filters"]
filters = list(filter_info.keys())

In [ ]:
data_df = filter_df_from_queryset(
    spec_model.objects.all(), r_star=False, scale_to=None
)
corpus = pd.concat([metadata_df, data_df], axis=1).copy()
wave_mapping = pd.Series(DERIVED_CAM_DICT[instrument]['filters'])

# define some spectop parameters
corpus['r56'] = corpus['R5'] / corpus['R6']
corpus['r16'] = corpus['R1'] / corpus['R6']
corpus['l645'] = ops.band_depth(
    corpus[['L6', 'L4', 'L5']].T,
    None,
    wave_mapping[['L6', 'L4', 'L5']]
)[0]
corpus['r153'] = ops.band_depth(
    corpus[['R1', 'R5', 'R3']].T,
    None,
    wave_mapping[['R1', 'R5', 'R3']]
)[0]
corpus['r465'] = ops.band_depth(
    corpus[['R4', 'R6', 'R5']].T,
    None,
    wave_mapping[['R4', 'R6', 'R5']]
)[0]

params = ['r56', 'r16', 'l645', 'r153', 'r465']
corpus = corpus.dropna(subset=params)


In [ ]:
# use this to to reduce to rock
# corpus = corpus.loc[corpus['feature']=='rock'].reset_index(drop=True)

# or other things, etc.
# corpus = corpus.loc[~corpus['feature'].isin(['crater rim', 'delta'])]

corpus = corpus.reset_index(drop=True)

log_r = LogisticRegression(
    multi_class='multinomial', max_iter=5000, class_weight='balanced'
)

# dependent qualitive variable to do a logistic regression against
dependent_name = 'feature'
dependent = corpus[dependent_name].dropna().unique()
exploded = explode_sequential(corpus, dependent_name)

# independent variables for the logistic regression
# independent = filters
independent = params

log_r.fit(corpus[independent], exploded)
log_coefficients = pd.DataFrame(
    log_r.coef_,
    columns = independent,
    index = dependent
)
score = log_r.score(corpus[independent], exploded)
morph_chart = make_corr_chart(log_coefficients)
set_label(
    morph_chart, 
    'zcam logit regression ' + str(score),
    fontproperties=mplf.FontProperties(size=24),
    x_or_y = 'x'
)

log_r.score(corpus[independent], exploded)

In [ ]:
# just make a correlation matrix out of the independent variables
ind_coefficients = correlation_matrix(corpus[independent])
plt.rcParams['font.size'] = 18
corrs = correlation_matrix(corpus[independent])
corrs = corrs.rename(
    columns={"incidence_angle": "\u03b8i"},
    index={"incidence_angle": "\u03b8i"},
)
corr_chart = make_corr_chart(corrs)
set_label(
    corr_chart, 
    'zcam correlation matrix',
    fontproperties=mplf.FontProperties(size=24),
    x_or_y = 'x'
)

In [ ]:
# save a figure
corr_chart.tight_layout()
corr_chart.savefig(
    'zcam_corr_matrix', dpi=220, bbox_inches="tight", pad_inches=0
)

In [ ]:
# save another figure
morph_chart.tight_layout()
morph_chart.savefig(
    'zcam_morph_logit', dpi=220, bbox_inches="tight", pad_inches=0
)
